# Course 4: Planning
## Part 2: Trajectory Generation
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Form a strong understanding of the basics of motion planning algorithms;
* Review the hybrid [A*](https://en.wikipedia.org/wiki/A*_search_algorithm) and its use in the Stanford Junior;
* Implement the hybrid [A*](https://en.wikipedia.org/wiki/A*_search_algorithm) search algorithm in Python and C++.

## 1. Introduction

### 1.1. Types of Motion Planning Algorithms

#### Background

In this lesson, we cover the basics of [motion planning](https://en.wikipedia.org/wiki/Motion_planning) algorithms; here we move from behaviour selection to full trajectory generation. In order to produce not only trajectory curves but also the time-sequence at which a robot executes these curves, we employ a variety of motion planning algorithms and direct our attention towards the areas of safety, feasibility, comfort and efficiency while generating vehicle trajectories.

#### Properties and problem set-up

Several important properties of motion planning algorithms are _completeness_ and _optimality_. The _completness_ assumption asserts that if a solution exists, the motion planner is guaranteed to always find the solution. In scenarios where no solution exists, the algorithm will correctly report so.  The _optimality_ assumption asserts that, for a given cost function evaluating sequences of actions, the motion planner is guaranteed to always return a feasible (i.e., executable) sequence of actions with minimal cost. Several other assumptions exist for specific algorithms, namely the resolution completeness and probabilistic completeness in the case of sampling-based planning algorithms.  

In order to enforce these constraints, we need a well-defined problem — the [_configuration space_](https://en.wikipedia.org/wiki/Configuration_space_\(physics\)) defines all possible configurations a robot can make in the world coordinate frame. In a 2D environment, this corresponds to each and every position $\left[x, y\right]$. In 3D, we extend the 2D position to include an orientation angle $\left[x, y, \theta\right]$. We must also define a _start_ and _goal configuration_; the start configuration $q_{\mathrm{start}}$ is usually initialised from a localisation algorithm using sensor data, whereas the goal configuration $q_{\mathrm{goal}}$ is selected w.r.t. the behaviour planner defining where to move and at which speed. In practise, motion planning problem definitions usually consider additional constraints, such as the sequence of actions predicted for non-vehicles and pedestrians, which can be obtained via map and traffic data as well as kinematic / physics-based models.

#### Types of motion planning algorithms

Several well-known types motion planning algorithms are as follows:
* [Combinatorial methods](http://msl.cs.uiuc.edu/planning/ch6.pdf): combinatorial approaches to motion planning find paths through the continuous configuration space without resorting to approximations — for special classes such as 2D robot planning, combinatorial methods provide _complete_, efficient, approximation-independent solutions over sampling-based planning methods [1];
    * Relies on a _roadmap_ assumption through cell decomposition (i.e., partitioning the search space into cells) and several properties regarding accessibility and connectivity of the discrete graph search sapce; 
* [Artificial potential fields](https://en.wikipedia.org/wiki/Motion_planning#Artificial_potential_fields): this type of reactive method defines obstacles as electrostatic point charges which generate potential fields — the robot navigates the anti-gravity field around e.g., pedestrians, bicyclists, using a [navigation function](https://en.wikipedia.org/wiki/Navigation_function).
    * A disadvantage with this class of algorithms (aside from harmonic potential field-based implementations) is their sensitivity to local minima, which might prevent the planner from arriving at the intended goal state;
* [Optimal control](https://en.wikipedia.org/wiki/Optimal_control): a set of methods for [trajectory optimisation](https://en.wikipedia.org/wiki/Trajectory_optimization) using either [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) and principle of optimality assumption (in closed-loop), or [calculus of variations](https://en.wikipedia.org/wiki/Calculus_of_variations) (in open-loop);
    * Solves both the motion planning and control input generation problems in one approach using dynamic vehicle model with a start- and end configuration;
    * Here a sequence of control inputs (e.g., steering angle, throttle) are generated and optimised with a cost function relative to the configuration of the vehicle (e.g., maximising efficiency, maintaining safe distance to obstacles);
    * Optimal control relies on numerical approximation methods, or [reinforcement learning](https://en.wikipedia.org/wiki/Reinforcement_learning)-based approaches for closed-loop control;
* [Sampling-based methods](https://en.wikipedia.org/wiki/Motion_planning#Sampling-based_algorithms): unlike combinatorial approaches, these set of algorithms have a runtime that is not explicitly exponentially dependent on the dimensionality of the configuration space $C$ — sampling-based algorithms construct roadmaps using a less-elegant collision dectection test which is _probabilistically complete_. In other words, the probability of a solution approaches $1.0$ as the free search space is exahusted;
    * For a cell decomposition / discretisation approach, [graph-based search](https://en.wikipedia.org/wiki/Graph_traversal) algorithms (e.g., [A*](https://en.wikipedia.org/wiki/A*_search_algorithm), [Dijkstra's](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm), [D* / D* Lite](https://en.wikipedia.org/wiki/D*), [ARA*](https://en.wikipedia.org/wiki/Anytime_A*), etc.);
    * For probabilistc methods, i.e., random exploration of configuration and input space, [probabilistic data structures](https://en.wikipedia.org/wiki/Category:Probabilistic_data_structures) are used (e.g., [RRT / RRT*](https://en.wikipedia.org/wiki/Rapidly-exploring_random_tree#Variants), [PRM](https://en.wikipedia.org/wiki/Probabilistic_roadmap), etc.).

### 1.2. Hybrid A*

#### Background

Hybrid A* (also known as Hybrid-State A* Search [2]), is a path-planning algorithm adapting the well-known [A* search](https://en.wikipedia.org/wiki/A*_search_algorithm) to 3D kinematic state space of the vehicle. The Hybrid A* search algorithm uses a modified state-update rule to capture the continuous state of the vehicle in the discretised map space (represented as nodes in the A* graph). This guarantees kinematic feasability of the generated trajectory. In other words, Hybrid A* finds a path that a vehicle can actually manoeuvre (e.g., smooth curve-based trajectories) which is favoured by vehicle motion models such as the [kinematic bicycle](https://thomasfermi.github.io/Algorithms-for-Automated-Driving/Control/BicycleModel.html).

#### Applications in path planning

Unlike the standard A* algorithm, the Hybrid A* is *not* guaranteed to find an minimal-cost solution. It is not _complete_, meaning it does not always find a solution when one exists. Aside from this, Hybrid A* has been implemented in Stanford's _Junior_ [3], the [2007 DARPA Grand Challenge](https://en.wikipedia.org/wiki/DARPA_Grand_Challenge_\(2007\)) 2nd Place winner, to perform free-form navigation in abritrary environments (e.g., parking lots). In order to perform trajectory planning, the Hybrid A* algorithm explores / expands discrete states using continuous vehicle coordinates $<x, y, \theta>$. A prediction $<x^{\prime}, y^{\prime}, \theta^{\prime}>$ is made with respect to the vehicle heading, and the path selected by Hybrid A* is guided by two additional heuristics: _non-holonomic-without-obstacles_, and the _holonomic-with-obstacles_. In addition to the runtime-, behaviour-, manoeuvre execution- and obstacle distance-based costs of the standard A* search, the Stanford Junior team used these two differential constraints to account for the non-holonomic nature of the ego-vehicle w.r.t. its location, orientation and direction of motion. This heuristic aided Junior in approaching the goal state with the desired heading. The second differential motion heuristic, _holonomic-with-obstacles_, ignores the non-holonomic nature of the vehicle and emphasizes the shortest distance to the goal. This second heuristic is computed online via a dynamic programming algorithm in 2D, whereas the first heuristic is computed entirely offline for the entire 4D space (vehicle location, orientation, and direction of motion). The _non-holonomic-without-obstacles_ heuristic is said to be significantly more efficient than Euclidean distance as it takes into account vehicle orientation, however, this heuristic alone fails to produce executable trajectories in situations with U-shaped dead ends. The addition of the _holonomic-with-obstacles_ heuristic attempts to address this by discovering these obstacles and dead-ends in 2D, which is used to guide the more-expensive 3D search away from these areas [2].

The Hybrid A* search uses analytic expansions based on the [Reed-Shepp model](http://lavalle.pl/planning/node822.html). This addresses the precision issue resulting from the discretisation of the vehicle control action space. The effect of steering angle discretisation on precision means that the grid resolution must be carefully selected such that a decent approximation of the exact continuous-coordinate goal state can be reached. A node in the tree is expanded by simulating a kinematic model of the car for a given control action over the time-step equal to the resolution of the grid. Then, for some nodes, the optimal Reed-Shepp path is computed from the current- to goal-state (assuming no obstacle-free environment). By selectively applying the Reed-Shepp expansion, this extension of the search tree yields signficant benefits in both planning time and accuracy [2]. Several other path-cost ([Voronoi Field](https://en.wikipedia.org/wiki/Voronoi_diagram#Engineering)) and local optimisation / smoothing functions are applied to the final path planner used in the Stanford _Junior_. 

#### Implementation in Python

The Python implementation below outlines the [A* search]() algorithm using the [kinematic bicycle motion] model (i.e., the hybrid A* algorithm). The following variables and objects are used:
* `State(x, y, theta, g, f)`: An object which stores the `x`, `y`, position coordinates, heading angle `theta` (direction) and the current A* `g` and `f` values;
* `grid`: A binary-valued 2D array — here a value "1" indicates the presence of an obstacle, wheras a "0" corresponds to free (navigable) space;
* `SPEED`: The speed of the vehicle used in the bicycle motion model;
* `LENGTH`: The length of the vehicle used in the bicycle motion model;
* `NUM_THETA_CELLS`: The number of cells to divide a circle into — used to keep track of which states have already been visited. 

The bulk of the hybrid A* algorithm is defined within the `search` function. The `expand` function takes a `State` instance and a `goal` (not implemented below), and returns a list of possible next-states for a range of steering angles. This function contains the implementation of the bicycle motion model and the A* heuristic function.

```python
def expand(
        state, 
        goal
):
    """Selects a trajectory using the bicycle motion model and heuristic function.
    
    :param state: Current position (`x`, `y`), orientation `theta` and heuristic `g`, `f` values.
    :param goal: Coordinates of the goal location. TODO.
    """
    
    next_states = []
    # Span the next-state space given by the vehicle heading (direction)
    for delta in range(-35, 40, 5):
        ### Begin the bicycle motion model
        delta_rad = deg_to_rad(delta)
        omega = SPEED / LENGTH * tan(delta_rad)
        next_x = state.x + SPEED * cos(theta)
        next_y = state.y + SPEED * sin(theta)
        next_theta = normalize(state.theta + omega)
        ### Compute the heuristic evaluation
        next_g += 1
        next_f += heuristic(next_x, next_y, goal)
        ### Create a next-state object
        state = State(next_x, next_y, next_theta, next_g, next_f)
        next_states.append(state)
    return next_states
    
def search(
        grid, 
        start, 
        goal
):
    """Performs the A* search with closing and opening / expansion of next-states.
    
    The vehicle heading (theta) search space is discretised, i.e., the heading
    interval [0, +/- 2*pi] is partitioned into `NUM_THETA_CELLS` such that the
    lower discretised `theta` values are assigned lower stack numbers, whereas
    the `higher` theta values closer to 2*pi are assigned to larger stack numbers.
    
    In order to perform A* search, we sort the remaining states in `states_opened`
    by lowest f-value; this is crucial to A* as the lowest f-value state indicates
    an efficient starting point for the search.
    
    :param grid: 2D discretised map of cells. TODO.
    :param start: State object to start the search from (i.e., 2D position and heading).
    :param goal: Coordinates of the goal location. TODO.
    """
    
    # Store the `State` instances to explore
    states_opened = []
    # Initialise the 3D map from the 2D grid and discretised heading angle
    states_closed = [
        [[0 for x in range(grid[0])] for y in range(len(grid))]
            for cell in range(NUM_THETA_CELLS)
    ]
    # Store the previous visited `State` instances
    states_came_from = [
        [[0 for x in range(grid[0])] for y in range(len(grid))]
        for cell in range(NUM_THETA_CELLS)
    ]
    # Create a new starting `State`
    x = start.x
    y = start.y
    theta = start.theta
    g = 0
    f = heuristic(start.x, start.y, goal)
    state = State(x, y, theta, g, f)
    states_opened.append(state)
    ### Discretising the theta search space
    stack_num = theta_to_stack_number(state.theta)
    states_closed[stack_num][idx(state.x)][idx(state.y)] = state
    ### Explore the remaining states in `states_opened`
    while states_opened:
        # First sort states by lowest f-value in order to start the search
        # Then, get the state with the lowest f-value
        states_opened.sort(key=lambda state: state.f)
        current = states_opened.pop(0)
        # Check if the goal has been reached
        if (
            (idx(current.x) == goal.x)
            and (idx(current.y) == goal.y)
        ):
            return path
       # Get the set of next-possible states
       next_states = expand(current, goal)
       for next_s in next_states:
           if not in_grid(next_s):
               # Skip any states not in grid dimensions
               continue
           # Check that this state has not previously been visited
           # and that no obstacle exists at this location
           stack_num = theta_to_stack_number(next_s.theta)
           if (
               (not states_closed[stack_num][idx(next_s.x)][idx(next_s.y)])
               and (idx(current.y) == goal.y)
           ):
               # Add the valid state to the explored states stack
               states_oppened.append(next_s)
               # Close the visited `State` tuple
               states_closed[stack_num][idx(next_s.x)][idx(next_s.y)] = 1
               # Add the visited state to the trajectory history
               states_came_from[stack_num][idx(next_s.x)][idx(next_s.y)] = current
 ```    

In the above Python code we have a near-complete implementation of the hybrid A* search, but note that the following functions are assumed to have been implemented outside of the provided code:
* `deg_to_rad`: Returns the heading angle converted from degrees to radians;
* `normalize()`: Returns the normalised vehicle heading (plus a shift amount `omega`);
* `in_grid`: Returns the boolean indicating whether the given state is within the dimensions of the discretised map;
* `index()`: Returns the corresponding index of the position coordinate in the grid space;
* `theta_to_stack_number()`: Returns the corresponding discretised theta angle from range [0, 2*pi] to stack space;
* `heuristic`: Computes the heuristic value of the evaluated state position relative to the goal state.

With the variables defined above the Python code-block, we have a complete definition of the hybrid A* search that we can use in Sect. 2.1. to implement in C++ code. 

### 1.3. Polynomial Trajectory Generation

#### Background

In order to design vehicle trajectories that are both comfortable and feasible, we use [polynomial](https://en.wikipedia.org/wiki/Polynomial) curves. These curves allow the vehicle to comfortably accelerate to a desired speed without introducing significant [jerk](https://en.wikipedia.org/wiki/Jerk_\(physics\)), which otherwise would result in an unpleasant and potentially dangerous driving experience.

#### Jerk-Minimising Trajectories

To design minimised jerk polynomial trajectories, we first make several assumptions about the environment and the coordinate systems used to describe vehicle displacement.

First, we assume that the environment the ego-vehicle operates in is _structured_ — i.e., driving lanes, speed limits, controlled intersections, etc. form structure in the driving environment and provide the trajectory planner with useful information a priori. This differs from the unstructured environment — parking lots, for example, where pre-defined rules regarding how to move (e.g., direction of traffic flow, lane boundaries, speed limits) about are not always available or adhered to by others.

Next, we move from the 2D Cartesian coordinate system to the [Frenet](https://fjp.at/posts/optimal-frenet/) coordinate system to describe vehicle position and movement in a more intuitive way. Movement along an arbitrary road surface is described in two dimensions: $d$-axis for side-to-side lateral movements and $s$-axis for longitudinal back-and-forth movements. If we let the centre of a given lane be the origin of our Frenet reference frame, we can represent curvy roads as straight paths. Any ego-vehicle offset in either the $d$- or $s$-axis will be relative to the refererence path followed by the ego-vehicle, and not just the absolute position in 2D Cartesian space. Therefore, we can more easily discern intended vehicle trajectories and design vehicle trajectories independent of the curvature of the road (assuming that a rough estimate of the road curvature is known a priori).

Using the individual components of the Frenet coordinate frame — the longitudinal and lateral axes — we separate the vehicle trajectories in either axis in order to execute them in a dynamic world reference frame. To create a trajectory, we define both a _start_ and _goal_ state in three dimensions: $s$, $d$ and $t$. The start (initial) state in the lateral direction is denoted $d_{i}$, while the goal (final) state is denoted $d_{f}$. The same applies for the longitudinal direction $s_{i}$, $s_{f}$, respectively. With this, we form our _boundary conditions_, the initial and final states described in the lateral and longitudinal directions. Here we use the three quantities _position_, _velocity_ and _acceleration_ to represent the boundary conditions / constraints, i.e., 
* $s_{i}, d_{i}$: Initial position in the longitudinal and lateral directions;
* $\dot{s}_{i}, \dot{d}_{i}$: Initial velocity in the longitudinal and lateral directions;
* $\ddot{s}_{i}, \ddot{d}_{i}$: Initial acceleration in the longitudinal and lateral directions;
* $s_{f}, d_{f}$: Final position in the longitudinal and lateral directions;
* $\dot{s}_{f}, \dot{d}_{f}$: Final velocity in the longitudinal and lateral directions;
* $\ddot{s}_{f}, \ddot{d}_{f}$: Final acceleration in the longitudinal and lateral directions.

With the above boundary conditions, we compute the minimum jerk trajectory with the above as coefficients of the expanded polynomial for the total jerk $d^3{s}/d^{3}t$, such that position $s(t) \in \left[0, t_{f}\right]$. From the total squared jerk equation,
$$
\begin{align}
\int_{0}^{t_{f}} \dddot{s} dt
\end{align}
$$

We form the polynomial of $s(t) = \alpha_{0} + \alpha_{1}t + \alpha_{2}t^{2} + \ldots + \alpha_{n}t^{n} + \ldots$ as the sum:

$$
\begin{align}
s(t) = \sum_{n=0}^{\infty} \alpha_{n}t^{n}
\end{align}
$$

and incorporating the realisation $d^{m}s/dt^{m} = 0, \forall m \geq 6$, we have the following expression for the minimum 1D jerk trajectory:
$$
\begin{align}
s(t) = \alpha_{0} + \alpha_{1}t + \alpha_{2}t^{2} + \alpha_{3}t^{3} + \alpha_{4}t^{4} + \alpha_{5}t^{5}
\end{align}
$$

with coefficients:
$$
\begin{align}
[s_{i}, \dot{s}_{i}, \ddot{s}_{i}, s_{f}, \dot{s}_{f}, \ddot{s}_{f}], \quad
[d_{i}, \dot{d}_{i}, \ddot{d}_{i}, d_{f}, \dot{d}_{f}, \ddot{d}_{f}].
\end{align}
$$

In the 1D case, we might make the assumption that the initial and final velocities of a trajectory are zero. For example, a lane change from initial state $s_{i} = d_{i} = 0$ to final state $s_{f} = 30, d_{f} = 10$ can result in a coefficient vector with only one non-zero value, the lateral displacement $s_{f} = 10$.

#### Deriving Polynomial Coefficients

Starting with our quintic polynomial,
$$
\begin{align}
s(t) = \alpha_{0} + \alpha_{1}t + \alpha_{2}t^{2} + \alpha_{3}t^{3} + \alpha_{4}t^{4} + \alpha_{5}t^{5}
\end{align}
$$

we take the first and second derivatives to get the equations with respect to velocity and acceleration:
$$
\begin{align}
\dot{s}(t) &= \alpha_{1} + 2\alpha_{2}t + 3\alpha_{3}t^{2} + 4\alpha_{4}t^{3} + 5\alpha_{5}t^{4}, \\
\ddot{s}(t) &= 2\alpha_{2} + 6\alpha_{3}t + 12\alpha_{4}t^{2} + 20\alpha_{5}t^{3}. \\
\end{align}
$$

Then, we set the initial time to zero, i.e., $t_{i} = 0$, and reduce the six unknowns into three. This results in:
$$
\begin{align}
s_{i} &= s(0) = \alpha_{0}, \\
\dot{s}_{i} &= \dot{s}(0) = \alpha_{1}, \\
\ddot{s}_{i} &= \ddot{s}(0) = 2\alpha_{2}. \\
\end{align}
$$

Gathering the unknowns into functions of the start boundary conditions, and assuming the constants $C_{i}$ from the original equations, we have:
$$
\begin{align}
s(t_{f}) &= s_{f} = \alpha_{3}t^{3}_{f} + \alpha_{4}t^{4}_{f} + \alpha_{5}t^{5}_{f} &+ C_{1}, \\
\dot{s}(t_{f}) &= \dot{s}_{f} = 3\alpha_{3}t^{2}_{f} + 4\alpha_{4}t^{3}_{f} + 5\alpha_{5}t^{4}_{f} &+ C_{2}, \\ 
\ddot{s}(t_{f}) &= \ddot{s}_{f} = 6\alpha_{3}t_{f} + 12\alpha_{4}t^{2}_{f} + 20\alpha_{5}t^{3}_{f} &+ C_{3}.
\end{align}
$$

Given that we know the final state values $\left[s_{f}, \dot{s}_{f}, \ddot{s}_{f}, t_{f}\right]$, we form the system of three equations as a matrix equation $A\mathrm{x} = b$:
$$
\begin{align}
\begin{bmatrix}
    t^{3}_{f} & t^{4}_{f} & t^{5}_{f} \\
    3t^{2}_{f} & 4t^{3}_{f} & 5t^{4}_{f} \\
    6t_{f} & 12t^{2}_{f} & 20t^{3}_{f} \\
\end{bmatrix} \times 
    \begin{bmatrix} \alpha_{3} \\ \alpha_{4} \\ \alpha_{5} \end{bmatrix}
    = \begin{bmatrix} s_{f} \\ \dot{s}_{f} \\ \ddot{s}_{f} \end{bmatrix}
        \begin{matrix} -C_{1} \\ -C_{2} \\ -C_{3}\end{matrix}
\end{align}
$$

This matrix equation allows us to solve for the minimum jerk trajectory coefficients needed to design the polynomial trajectories along both the $s$- and $d$-axis. Using a polynomial solver, we can obtain these coefficients for the longitudinal and lateral trajectories.

## 2. Programming Task

In [1]:
#include "4-2-Trajectory-Generation.h"

### 2.1. Hybrid A* in C++

In [2]:
// From J. Moran's `2_tests.cc`

In [3]:
#include "4-2-Trajectory-Generation/1_hybrid_breadth_first.h"

In [4]:
/* Tests the Hybrid A* algorithm implemented with breadth-first search.
 * 
 * Here the 2D grid environment is defined along with a 3D starting pose and
 * a 2D goal position. The Hybrid A* search finds the shortest path by
 * minimising the Manhattan distance cost from the current expansion node to
 * the goal position.
 * 
 * CANDO: Modify the maze environment (obstaces / free-space) or use different
 * cost / heuristic functions (e.g., holonomic-with-obstacles, Dolgov 2008).
 *
 */
void test_hybrid_breadth_first() {
  // Initialising the maze grid
  int X = 1;
  int _ = 0;
  // CANDO: Modify the grid maze to test different expansions.
  std::vector<std::vector<int>> GRID = {
    {_,X,X,_,_,_,_,_,_,_,X,X,_,_,_,_,},
    {_,X,X,_,_,_,_,_,_,X,X,_,_,_,_,_,},
    {_,X,X,_,_,_,_,_,X,X,_,_,_,_,_,_,},
    {_,X,X,_,_,_,_,X,X,_,_,_,X,X,X,_,},
    {_,X,X,_,_,_,X,X,_,_,_,X,X,X,_,_,},
    {_,X,X,_,_,X,X,_,_,_,X,X,X,_,_,_,},
    {_,X,X,_,X,X,_,_,_,X,X,X,_,_,_,_,},
    {_,X,X,X,X,_,_,_,X,X,X,_,_,_,_,_,},
    {_,X,X,X,_,_,_,X,X,X,_,_,_,_,_,_,},
    {_,X,X,_,_,_,X,X,X,_,_,X,X,X,X,X,},
    {_,X,_,_,_,X,X,X,_,_,X,X,X,X,X,X,},
    {_,_,_,_,X,X,X,_,_,X,X,X,X,X,X,X,},
    {_,_,_,X,X,X,_,_,X,X,X,X,X,X,X,X,},
    {_,_,X,X,X,_,_,X,X,X,X,X,X,X,X,X,},
    {_,X,X,X,_,_,_,_,_,_,_,_,_,_,_,_,},
    {X,X,X,_,_,_,_,_,_,_,_,_,_,_,_,_,}
  };
  std::vector<double> START = {0.0, 0.0, 0.0};
  std::vector<int> GOAL = {
    (int)GRID.size() - 1, 
    (int)GRID[0].size() - 1
  };
  std::cout << "Finding path through grid:" << "\n";
  // Creates an empty maze to test number of expansions
  for (int i = 0; i < GRID.size(); ++i) {
    std::cout << GRID[i][0];
    for (int j = 1; j < GRID[0].size(); ++j) {
      std::cout << "," << GRID[i][j];
    }
    std::cout << "\n";
  }
  HBF hbf = HBF();
  HBF::maze_path get_path = hbf.search(
    GRID,
    START,
    GOAL
  );
  std::vector<HBF::maze_s> show_path = hbf.reconstruct_path(
    get_path.states_came_from, 
    START, 
    get_path.final
  );
  std::cout << "Show path from start to finish" << "\n";
  for (int i = show_path.size()-1; i >= 0; --i) {
      HBF::maze_s step = show_path[i];
      std::cout << "##### step " << step.g << " #####" << "\n";
      std::cout << "x " << step.x << "\n";
      std::cout << "y " << step.y << "\n";
      std::cout << "theta " << step.theta << "\n";
  }
}

#### Testing the Hybrid A* with BFS

In [5]:
// From J. Moran's `2_tests.cc`

In [6]:
test_hybrid_breadth_first()

Finding path through grid:
0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0
0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0
0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0
0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,0
0,1,1,0,0,0,1,1,0,0,0,1,1,1,0,0
0,1,1,0,0,1,1,0,0,0,1,1,1,0,0,0
0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0
0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0
0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0
0,1,1,0,0,0,1,1,1,0,0,1,1,1,1,1
0,1,0,0,0,1,1,1,0,0,1,1,1,1,1,1
0,0,0,0,1,1,1,0,0,1,1,1,1,1,1,1
0,0,0,1,1,1,0,0,1,1,1,1,1,1,1,1
0,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Found path to goal in 9526 expansions
Show path from start to finish
##### step 1 #####
x 1.45
y 0
theta 0
##### step 2 #####
x 2.9
y 0
theta 0
##### step 3 #####
x 4.35
y 0
theta 0.511348
##### step 4 #####
x 5.61452
y 0.709563
theta 6.01748
##### step 5 #####
x 7.01364
y 0.328808
theta 6.52883
##### step 6 #####
x 8.42011
y 0.681421
theta 6.27511
##### step 7 #####
x 9.87007
y 0.669715
theta 7.6274
##### step 8 #####
x 10.1958
y 2.08265
theta 7.37369
####

### 2.2. Quintic Polynomial Solver in C++

In this exercise we solve the matrix equation from Sect. 1.2. using C++. Recall the matrix equation of the form $A\mathrm{x} = b$:
$$
\begin{align}
\begin{bmatrix}
    t^{3}_{f} & t^{4}_{f} & t^{5}_{f} \\
    3t^{2}_{f} & 4t^{3}_{f} & 5t^{4}_{f} \\
    6t_{f} & 12t^{2}_{f} & 20t^{3}_{f} \\
\end{bmatrix} \times 
    \begin{bmatrix} \alpha_{3} \\ \alpha_{4} \\ \alpha_{5} \end{bmatrix}
    = \begin{bmatrix}
        s_{f} - \left(s_{i} + \dot{s}_{i})t + \frac{1}{2}\ddot{s}_{i}t^{2}\right) \\ 
        \dot{s}_{f} - \left(\dot{s}_{i} + \ddot{s}_{i}t\right) \\ 
        \ddot{s}_{f} - \ddot{s}_{i} \end{bmatrix}
\end{align}
$$

for the 1D kinematic equations evaluated at $t = 0$. This yields the first set of knowns $\left[\alpha_{0}, \alpha_{1}, \alpha_{2}\right] = \left[s_{i}, \dot{s}_{i}, \frac{1}{2}\ddot{s}_{i}\right]$. The last three coefficients $\left[\alpha_{3}, \alpha_{4}, \alpha_{5}\right]$ in the matrix equation above can be solved by evaluating the 1D kinematic equations for $t = t_{f}$, as carried out in the above.

In order to solve this quintic polynomial, we use the [Eigen](https://en.wikipedia.org/wiki/Eigen_(C%2B%2B_library)) C++ linear algebra library. To solve the system of the form $A\mathrm{x} = b$, we multiply the vector $b$ by the matrix inverse of $A$ (assuming it exists). Knowing that the first three coefficients are given as $\left[\alpha_{0}, \alpha_{1}, \alpha_{2}\right] = \left[s_{i}, \dot{s}_{i}, \frac{1}{2}\ddot{s}_{i}\right]$, we compute the last three unknowns $\left[\alpha_{3}, \alpha_{4}, \alpha_{5}\right]$ with the relation:
$$
\begin{align}
\mathrm{x} &= A^{-1}b.
\end{align}
$$

In [7]:
// From J. Moran's `2_tests.cc`

In [8]:
#include "4-2-Trajectory-Generation/2_jerk_minimising_trajectory.h"

In [9]:
/* Tests the quintic polynomial solver using the test cases in `2_grader.h`.
 *
 * The `JMT` function in `2_jerk_minimising_trajectory.cc` returns the six
 * coefficient values corresponding to the quintic polynomial of the minimised
 * jerk trajectory. The matrix equation $Ax = b$ is solved using the inverse of
 * the matrix $A$ which is assumed to exist.
 * 
 * This helper function tests the returned coefficient values against the
 * expected values up to a deviation amount `epsilon` defined in `2_grader.h`.
 */
void test_jerk_minimising_trajectory() {
  // Create the test cases
  std::vector<test_case> tc = create_tests();
  bool total_correct = true;
  for (int i = 0; i < tc.size(); ++i) {
    std::vector<double> jmt = JMT(
        tc[i].start,
        tc[i].end,
        tc[i].T
    );
    bool correct = close_enough(jmt, answers[i]);
    total_correct &= correct;
  }
  if (!total_correct) {
    std::cout << "Try again!" << "\n";
  }
  else {
    std::cout << "Nice work!" << "\n";
  }
}

#### Testing the Quintic Polynomial Solver in C++

In [10]:
test_jerk_minimising_trajectory()

Nice work!


## Credits

This lesson has been prepared by Sebastian Thrun and Emmanuel Boidot, 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).

References
* [1] LaValle, S. M. Combinatorial Motion Planning. In: Planning Algorithms. Cambridge University Press. 2006. pp.206-256. [doi:10.1017/CBO9780511546877.008]().ISBN-13: 978-0521862059.
* [2] Dolgov, D. et al. Practical Search Techniques in Path Planning for Autonomous Driving. Association for the Advancement of Artificial Intelligence. Pre-print. 2008. [https://www.aaai.org/Library/Workshops/2008/ws08-10-006.php](https://www.aaai.org/Papers/Workshops/2008/WS-08-10/WS08-10-006.pdf).
* [3] Montemerlo, M. et al. Junior: The Stanford Entry in the Urban Challenge. Journal of Field Robotics. 2008. 25(9):569-597. [doi:10.1002/rob.20258](https://doi.org/10.1002/rob.20258).

Helpful resources:
* [Introduction to Optimal Control by Sumeet Singh | Princeton University ORF523 Lecture Notes](https://www.princeton.edu/~aaa/Public/Teaching/ORF523/ORF523_S21_Guest_Lecture.pdf)